# Código principal

Nesse código é implementado todo o pipeline proposto pelo projeto, como definido na imagem a seguir:

![image](../diagramaPI.drawio.png)

## Instalação das bibliotecas necessárias

In [1]:
%pip install numpy
%pip install pandas
%pip install pytesseract
%pip install opencv-python
%pip install matplotlib
%pip install imutils
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Importações de bibliotecas externas

In [2]:
import pytesseract
import ipywidgets as widgets

Para configurar o `pytesseract` é preciso instalar o seu executável. Se não tiver configurado ele nas variáveis de ambiente, modificar a variável do código abaixo para indicar o caminho até o executável do `tesseract`.

A documentação do `pytesseract` pode ser [acessada aqui](https://pypi.org/project/pytesseract/)

O `tesseract` pode ser instalado [por aqui](https://tesseract-ocr.github.io/tessdoc/Installation.html)

In [3]:
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

## Importações internas

In [4]:
from image_preprocessing.image_preprocessing import preprocess_image
from image_preprocessing.document_digitalization import mark_document_found

from OCR.image_to_dataframe import image_to_dataframe

from relevant_info_identification.relevant_info_identification import get_document_info_and_mark_relevant_regions

## Criando objetos da interface gráfica do projeto

In [5]:
%%html
<style>
.selected{
    background-color: rgb(0, 255, 0);
}
.not-selected{
    background-color: rgb(255, 0, 0);
}
</style>

In [6]:
original_image_widget = widgets.Image(
    format='jpg',
    width=600,
    height=400
)


process_btn = widgets.Button(
    description='Processar',
    button_style='success',
    icon='check'
)

stop_btn = widgets.Button(
    description='Parar',
    button_style='danger',
    icon='times'
)


no_rotate_marked_image_widget = widgets.Image(
    format='jpg',
    width=600,
    height=400
)

no_rotate_infos_json_widget = widgets.Text()

no_rotate_prob_widget = widgets.Text()


rotate_180_marked_image_widget = widgets.Image(
    format='jpg',
    width=600,
    height=400
)

rotate_180_infos_json_widget = widgets.Text()

rotate_180_prob_widget = widgets.Text()


no_rotate_box = widgets.VBox([
    no_rotate_marked_image_widget,
    no_rotate_infos_json_widget,
    no_rotate_prob_widget
])
no_rotate_box.add_class('selected')


rotate_180_box = widgets.VBox([
    rotate_180_marked_image_widget,
    rotate_180_infos_json_widget,
    rotate_180_prob_widget
])
rotate_180_box.add_class('not-selected')
pass

## Definindo função do pipeline completo

## Mostrando interface

In [7]:
display(original_image_widget)

display(widgets.HBox([
    no_rotate_box,
    rotate_180_box
]))

Image(value=b'', format='jpg', height='400', width='600')